# Getting features as structured data from tiled segmentation data

In [2]:
from Cells import Cells
from MGFeatures import *

In [ ]:
folder = r'D:\Hanyi temp\MG\cell-png'

cell5 = Cells(5)
imageshape = cell5.get_imageshape(folder)
cell5_image = cell5.img_from_tiles(folder)
cell5_labels = cell5.label_cells(cell5_image)
cell5_bboxes = cell5.get_bbox(cell5_labels)

In [ ]:
heterochromatin_folder = r'X:\Hanyi\MG\heterochromatin'
lipids_folder = r'X:\Hanyi\MG\lipids'
lysosomes_folder = r'X:\Hanyi\MG\lysosomes'
mitochondria_folder = r'X:\Hanyi\MG\mitochondria'
nucleus_folder = r'X:\Hanyi\MG\nucleus'
organelles_folder = r'X:\Hanyi\MG\organelles_cyto'
ER_folder = r'X:\Hanyi\MG\ER'


def make_properties(i):
    properties = {}
    
    # label cells
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) #TODO: this step is redundant! Fix in cells
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    cell_bboxes = cell.get_bbox(cell_labels)
    
    
    # label organelles
    heterochromatin_labels = mask_it(cell_labels, cell.img_from_tiles(heterochromatin_folder))
    lipids_labels = mask_it(cell_labels, cell.img_from_tiles(lipids_folder))
    lysosomes_labels = mask_it(cell_labels, cell.img_from_tiles(lysosomes_folder))
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    organelles_labels = mask_it(cell_labels, cell.img_from_tiles(organelles_folder))
    ER = cell.img_from_tiles(ER_folder)
    
    
    # calc properties
    labels_list = np.unique(cell_labels)[1:]

    properties['cell_area'] = get_area(cell_labels, labels_list)
    properties['organelles_area'] = get_area(organelles_labels, labels_list)
    properties['nucleus_area'] = get_area(nucleus_labels, labels_list)
    properties['heterochromatin_area'] = get_area(heterochromatin_labels, labels_list)
    properties['lipids_area'] = get_area(lipids_labels, labels_list)
    properties['mitochondria_area'] = get_area(mitochondria_labels, labels_list)
    properties['lysosomal_area'] = get_area(lysosomes_labels, labels_list)
    
    properties['empty_area'] = properties['cell_area'] - properties['nucleus_area'] - properties['organelles_area']
    properties['cytoplasm_area'] = properties['cell_area'] - properties['nucleus_area']
    
    properties['ER_lengths'] = ER_length(ER, cell_labels)
    
    properties['sl_num'] = np.array([i]*len(labels_list))
    
    properties['bbox_rects'] = cell_bboxes
    
    return properties

In [ ]:
nslices = 17
properties = Parallel(n_jobs=nslices, verbose=True)(delayed(make_properties)(i) for i in range(nslices))
features = [pd.DataFrame.from_dict(properties[i]) for i in range(nslices)]
_featuresall = pd.concat([features], ignore_index=True)
_featuresall.to_excel('featuresraw_v05.xlsx', index=False)